In the code you will find the following:
    
1. EDA of the continous and categorical variable
2. Weight of Evidence(WOE) and Information value(IV) for continous and Categorical Feature
3. Modelling using Boosting techniques and WOE transformed features   

To study more about WOE, you can refer to below links:
1. https://www.kaggle.com/ankarsingh/credit-card-lead-prediction-using-woe
2. https://www.listendata.com/2015/03/weight-of-evidence-woe-and-information.html

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
import pandas.core.algorithms as algos
from pandas import Series
import scipy.stats.stats as stats
import re
import string
import warnings
warnings.filterwarnings('ignore')
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer
from sklearn.metrics import f1_score, confusion_matrix,roc_auc_score, roc_curve, auc, \
            classification_report, recall_score, precision_recall_curve
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import accuracy_score, f1_score
from catboost import CatBoostClassifier
from sklearn.model_selection import train_test_split, StratifiedKFold
import shap
from sklearn import preprocessing
shap.initjs()

import os# data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory


for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
train = pd.read_csv(r'/kaggle/input/titanic/train.csv')
test = pd.read_csv(r'/kaggle/input/titanic/test.csv')

In [ ]:
train.head()

**EDA**

In [ ]:
# plot countplots 
cat_col = ['Sex','SibSp','Embarked','Parch','Pclass']
plt.figure(figsize=(14, 12), dpi=100)
for i, feature in enumerate(cat_col):
    plt.subplot(3, 3, i+1)
    sns.countplot(data=train, x=feature)
    
sns.despine()

In [ ]:
# plot countplots 
cat_col = ['Sex','SibSp','Embarked','Parch','Pclass']
plt.figure(figsize=(14, 12), dpi=100)
for i, feature in enumerate(cat_col):
    plt.subplot(3, 3, i+1)
    sns.countplot(data=train, x=feature, hue='Survived')
    
sns.despine()

In [ ]:
sns.distplot(train["Age"])



In [ ]:
sns.distplot(train["Fare"])#skewed

In [ ]:
train.isna().sum()

**Feature Engineering**

In [ ]:
#Treating Missing values  in Age and Embarked, Cabin will be treated by WOE
train['Age'] = train['Age'].fillna(train['Age'].median())
train['Embarked'] = train['Embarked'].fillna(train['Age'].mode())

test['Age'] = test['Age'].fillna(train['Age'].median())
test['Embarked'] = test['Embarked'].fillna(train['Age'].mode())

In [ ]:
#Calculatiing number of relatives
train['relatives'] = train['SibSp'].astype(int) + train['Parch'].astype(int)
test['relatives'] = test['SibSp'].astype(int) + test['Parch'].astype(int)


In [ ]:
train['title'] = train.Name.str.extract(r',\s*([^\.]*)\s*\.', expand=False)
train['title'].unique()

In [ ]:
titles = ['Mr', 'Mrs', 'Miss', 'Master','Dr','Ms','Lady', 'Sir','Col', 'Capt','Major']
train.loc[~train['title'].isin(titles),'title'] = 'Others'

In [ ]:
#As 'Dr','Ms','Lady', 'Sir','Col', 'Capt','Major' have low count lets put them also in others
train['title'].value_counts()

In [ ]:
titles = ['Mr', 'Mrs', 'Miss', 'Master','Dr']
train.loc[~train['title'].isin(titles),'title'] = 'Others'

In [ ]:
train['title'].value_counts()

In [ ]:
sns.countplot(data=train, x='title', hue='Survived')

In [ ]:
#Apply same thing on test
test['title'] = test.Name.str.extract(r',\s*([^\.]*)\s*\.', expand=False)
test.loc[~test['title'].isin(titles),'title'] = 'Others'

In [ ]:
test['title'].value_counts()

In [ ]:
le = preprocessing.LabelEncoder()
train['title_encoder'] = le.fit_transform(train['title'])
test['title_encoder'] = le.transform(test['title'])

In [ ]:
max_bin = 20
force_bin = 3

# Binning Function for continous variables
def mono_bin(Y, X, n = max_bin):
    
    df1 = pd.DataFrame({"X": X, "Y": Y})
    justmiss = df1[['X','Y']][df1.X.isnull()]
    notmiss = df1[['X','Y']][df1.X.notnull()]
    r = 0
    while np.abs(r) < 1:
        try:
            d1 = pd.DataFrame({"X": notmiss.X, "Y": notmiss.Y, "Bucket": pd.qcut(notmiss.X, n)})
            d2 = d1.groupby('Bucket', as_index=True)
            r, p = stats.spearmanr(d2.mean().X, d2.mean().Y)
            n = n - 1 
        except Exception as e:
            n = n - 1

    if len(d2) == 1:
        n = force_bin         
        bins = algos.quantile(notmiss.X, np.linspace(0, 1, n))
        if len(np.unique(bins)) == 2:
            bins = np.insert(bins, 0, 1)
            bins[1] = bins[1]-(bins[1]/2)
        d1 = pd.DataFrame({"X": notmiss.X, "Y": notmiss.Y, "Bucket": pd.cut(notmiss.X, np.unique(bins),include_lowest=True)}) 
        d2 = d1.groupby('Bucket', as_index=True)
    
    d3 = pd.DataFrame({},index=[])
    d3["MIN_VALUE"] = d2.min().X
    d3["MAX_VALUE"] = d2.max().X
    d3["COUNT"] = d2.count().Y
    d3["EVENT"] = d2.sum().Y
    d3["NONEVENT"] = d2.count().Y - d2.sum().Y
    d3=d3.reset_index(drop=True)
    
    if len(justmiss.index) > 0:
        d4 = pd.DataFrame({'MIN_VALUE':np.nan},index=[0])
        d4["MAX_VALUE"] = np.nan
        d4["COUNT"] = justmiss.count().Y
        d4["EVENT"] = justmiss.sum().Y
        d4["NONEVENT"] = justmiss.count().Y - justmiss.sum().Y
        d3 = d3.append(d4,ignore_index=True)
    
    d3["EVENT_RATE"] = d3.EVENT/d3.COUNT
    d3["NON_EVENT_RATE"] = d3.NONEVENT/d3.COUNT
    d3["DIST_EVENT"] = d3.EVENT/d3.sum().EVENT
    d3["DIST_NON_EVENT"] = d3.NONEVENT/d3.sum().NONEVENT
    d3["WOE"] = np.log(d3.DIST_EVENT/d3.DIST_NON_EVENT)
    d3["IV"] = (d3.DIST_EVENT-d3.DIST_NON_EVENT)*np.log(d3.DIST_EVENT/d3.DIST_NON_EVENT)
    d3["VAR_NAME"] = "VAR"
    d3 = d3[['VAR_NAME','MIN_VALUE', 'MAX_VALUE', 'COUNT', 'EVENT', 'EVENT_RATE', 'NONEVENT', 'NON_EVENT_RATE', 'DIST_EVENT','DIST_NON_EVENT','WOE', 'IV']]       
    d3 = d3.replace([np.inf, -np.inf], 0)
    d3.IV = d3.IV.sum()
    
    return(d3)

# Binning Function for Categorical variables
def char_bin(Y, X):
        
    df1 = pd.DataFrame({"X": X, "Y": Y})
    justmiss = df1[['X','Y']][df1.X.isnull()]
    notmiss = df1[['X','Y']][df1.X.notnull()]    
    df2 = notmiss.groupby('X',as_index=True)
    
    d3 = pd.DataFrame({},index=[])
    d3["COUNT"] = df2.count().Y
    d3["MIN_VALUE"] = df2.sum().Y.index
    d3["MAX_VALUE"] = d3["MIN_VALUE"]
    d3["EVENT"] = df2.sum().Y
    d3["NONEVENT"] = df2.count().Y - df2.sum().Y
    
    if len(justmiss.index) > 0:
        d4 = pd.DataFrame({'MIN_VALUE':np.nan},index=[0])
        d4["MAX_VALUE"] = np.nan
        d4["COUNT"] = justmiss.count().Y
        d4["EVENT"] = justmiss.sum().Y
        d4["NONEVENT"] = justmiss.count().Y - justmiss.sum().Y
        d3 = d3.append(d4,ignore_index=True)
    
    d3["EVENT_RATE"] = d3.EVENT/d3.COUNT
    d3["NON_EVENT_RATE"] = d3.NONEVENT/d3.COUNT
    d3["DIST_EVENT"] = d3.EVENT/d3.sum().EVENT
    d3["DIST_NON_EVENT"] = d3.NONEVENT/d3.sum().NONEVENT
    d3["WOE"] = np.log(d3.DIST_EVENT/d3.DIST_NON_EVENT)
    d3["IV"] = (d3.DIST_EVENT-d3.DIST_NON_EVENT)*np.log(d3.DIST_EVENT/d3.DIST_NON_EVENT)
    d3["VAR_NAME"] = "VAR"
    d3 = d3[['VAR_NAME','MIN_VALUE', 'MAX_VALUE', 'COUNT', 'EVENT', 'EVENT_RATE', 'NONEVENT', 'NON_EVENT_RATE', 'DIST_EVENT','DIST_NON_EVENT','WOE', 'IV']]      
    d3 = d3.replace([np.inf, -np.inf], 0)
    d3.IV = d3.IV.sum()
    d3 = d3.reset_index(drop=True)
    
    return(d3)
def comb_category(woe,threshold):
    
    count = 0
    similar_col = dict()
    col = []
    columns1 = woe['MIN_VALUE'].unique()
    columns2 = woe['MIN_VALUE'].unique()
    for cat1 in columns1 :
        if cat1 in col: continue
        woe1 = float(woe[woe['MIN_VALUE'] == cat1]['WOE'].values[0])
        col1 = []

        for cat in woe['MIN_VALUE'].unique():
            if cat1 == cat: continue
            if cat in col: continue
            woe2 = float(woe[woe['MIN_VALUE'] == cat]['WOE'].values[0])

            if (woe2 - woe1) >0.0 and (woe2 - woe1)<threshold:
                col1.append(cat)
                col.append(cat)
        col.append(cat1)

        similar_col[cat1] = col1
        
        if len(col1)>0:
            count+=1
    
    return(similar_col,count)
def data_vars(df1,target,test,cat_threshold):
    
    
    
    x = df1.dtypes.index
    y = train_copy.Survived.name
    count = -1
    replace = {}
    
    for i in x:
        if i == y:continue
        if np.issubdtype(df1[i], np.number) and len(Series.unique(df1[i])) > 2:
            print('Numerical_col:',i)
            conv = mono_bin(target, df1[i])
            conv["VAR_NAME"] = i
            count = count + 1
        else:
            print('Categorical_col:',i)
            cat_replace = []
            conv = char_bin(target, df1[i])
            conv = conv.sort_values('WOE')
            similar_col,var_count = comb_category(conv,cat_threshold)
            cat_replace.append(similar_col)

            while var_count>0:

                for x,y in zip(similar_col.keys(),similar_col.values()):
                    df1.loc[df1[i].isin(y),i] = x
                    test.loc[test[i].isin(y),i] = x
                conv = char_bin(target, df1[i])
                conv = conv.sort_values('WOE')
                similar_col,var_count = comb_category(conv,cat_threshold)
                cat_replace.append(similar_col)
            replace[i] = cat_replace
            conv["VAR_NAME"] = i            
            count = count + 1

        if count == 0:
            iv_df = conv
        else:
            iv_df = iv_df.append(conv,ignore_index=True)
    
    iv = pd.DataFrame({'IV':iv_df.groupby('VAR_NAME').IV.max()})
    iv = iv.reset_index()
    return(iv_df,iv,df1,test,replace)



In [ ]:
train.head()

In [ ]:
train.isna().sum()

In [ ]:
train_copy = train.copy()
test_copy  = test.copy()

train_copy.drop(['PassengerId','Name'],axis =1,inplace = True)
train_copy['Cabin'].fillna('NA',inplace = True)
train_copy['Embarked'].fillna('NA',inplace = True)
train_copy['Ticket'].fillna('NA',inplace = True)

test_copy.drop(['PassengerId','Name'],axis =1,inplace = True)
test_copy['Cabin'].fillna('NA',inplace = True)
test_copy['Embarked'].fillna('NA',inplace = True)
test_copy['Ticket'].fillna('NA',inplace = True)

#Calculating WOE and IV values
final_iv, IV,new_train,new_test,cat_replace = data_vars(train_copy,train_copy.Survived,test_copy,cat_threshold = 0.1)

In [ ]:
final_iv.head()

In [ ]:
def woe_replacement(train,transform_vars_list,transform_prefix):
    for var in transform_vars_list:
        print(var)
        
        small_train = final_iv[final_iv['VAR_NAME'] == var]
        transform_dict = dict(zip(small_train.MAX_VALUE,small_train.WOE))
        replace_cmd = ''
        replace_cmd1 = ''
        
        for i in sorted(transform_dict.items()):
            replace_cmd = replace_cmd + str(i[1]) + str(' if x <= ') + str(i[0]) + ' else '
            replace_cmd1 = replace_cmd1 + str(i[1]) + str(' if x == "') + str(i[0]) + '" else '
        replace_cmd = replace_cmd + '0'
        replace_cmd1 = replace_cmd1 + '0'
        
        if replace_cmd != '0':
            try:
                train[transform_prefix + var] = train[var].apply(lambda x: eval(replace_cmd))
            except:
                train[transform_prefix + var] = train[var].apply(lambda x: eval(replace_cmd1))
                
    return(train)

In [ ]:
transform_vars_list = new_train.columns.difference(['Survived'])
transform_prefix = 'new_'

new_train = woe_replacement(new_train,transform_vars_list,transform_prefix)

In [ ]:
transform_vars_list = new_test.columns
transform_prefix = 'new_'

new_test = woe_replacement(new_test,transform_vars_list,transform_prefix)

**Modelling**

In [ ]:
def boosting(clf, fit_params, train, test, features):
    N_SPLITS = 3
    oofs = np.zeros(len(train))
    preds = np.zeros((len(test)))

    folds = StratifiedKFold(n_splits = N_SPLITS)

    for fold_, (trn_idx, val_idx) in enumerate(folds.split(train, train[TARGET_COL])):
        print(f'\n------------- Fold {fold_ + 1} -------------')

        ### Training Set
        X_trn, y_trn = train[features].iloc[trn_idx], train[TARGET_COL].iloc[trn_idx]

        ### Validation Set
        X_val, y_val = train[features].iloc[val_idx], train[TARGET_COL].iloc[val_idx]

        ### Test Set
        X_test = test[features]

        #print(X_trn)
        #exit(0)

        _ = clf.fit(X_trn, y_trn, eval_set = [(X_val, y_val)], **fit_params)
        
#         shap_values = shap.TreeExplainer(clf).shap_values(X_trn)
#         shap.summary_plot(shap_values,X_trn, plot_type = 'bar')

        ### Instead of directly predicting the classes we will obtain the probability of positive class.
        preds_val = clf.predict_proba(X_val)[:, 1]
        preds_test = clf.predict_proba(X_test)[:, 1]

        roc_score = roc_auc_score(y_val,preds_val)
        print("ROC for validation set is {}".format(roc_score))

        oofs[val_idx] = preds_val
        preds += preds_test / N_SPLITS


    oofs_score = roc_auc_score(train[TARGET_COL], oofs.round())
    print('ROC score for oofs is {}'.format(oofs_score))


    return oofs, preds,clf

In [ ]:
new_train.columns

In [ ]:
#catboost model training
clf = CatBoostClassifier(n_estimators = 200,
                       learning_rate = 0.05,
                       rsm = 0.2, ## Analogous to colsample_bytree
                       random_state=2054,
                         max_depth =6
                     )

fit_params = {'verbose': 200, 'early_stopping_rounds': 20}

features = ['new_Cabin','Age',
       'new_Embarked', 'new_Fare', 'Parch', 'Pclass', 'new_Sex',
       'SibSp', 'new_Ticket', 'relatives','title_encoder']

TARGET_COL = 'Survived'

cb_oofs, cb_preds,model = boosting(clf, fit_params,new_train,new_test,features)

optimized_roc = roc_auc_score(new_train[TARGET_COL], (cb_oofs  * 1))
print(f'Optimized ROC is {optimized_roc}')


In [ ]:
#Feature_importance
shap_values = shap.TreeExplainer(model).shap_values(new_train[features])
shap.summary_plot(shap_values,new_train[features], plot_type = 'bar')

In [ ]:
results = pd.DataFrame({'PassengerId':test['PassengerId'].values,'Survived':cb_preds})
results.loc[results['Survived']>0.5,'Survived'] = 1
results.loc[results['Survived']<=0.5,'Survived'] = 0

results['Survived'] = results['Survived'].astype(int)
results.to_csv(r'submission_catboost_v1.csv',index=False)